In [28]:
import numpy as np
np.set_printoptions(suppress=True)
import math
import matplotlib.pyplot as plt
import sympy
from matplotlib.patches import Ellipse, Arc
from matplotlib import cm, colors, patches

In [27]:
def Draw_truss(Nodes,Element,TotalNodes,TotalElements,x_min,x_max,y_min,y_max,R,mr):
    fig = plt.figure(figsize=(12,9))
    ax = fig.add_subplot(1,1,1)
    L1,L2 = (x_max-x_min)/6,(y_max-y_min)/6
    L = min(L1,L2)
    
    for i in range(TotalElements):
        n1,n2 = Element[i]['From_To'][0], Element[i]['From_To'][1]
        X = [Nodes[n1]['coordinates'][0],Nodes[n2]['coordinates'][0]]
        Y = [Nodes[n1]['coordinates'][1],Nodes[n2]['coordinates'][1]]
        ax.plot(X,Y,linewidth=1,color='b')
        
        ### ELEMENT ###
        x0,y0 = (X[0]+X[1])/2,(Y[0]+Y[1])/2
        ax.text(x0-0.1*L1, y0+0.1*L2, "E"+str(i+1))
    
    if mr==1:
        print("\nExternal Forces\n")
    if mr==2:
        print("\nReaction Forces\n")
    
    for i in range(TotalNodes):
        x,y = Nodes[i]['coordinates'][0],Nodes[i]['coordinates'][1]
        
        if Nodes[i]['joint'] == 'h':
            ax.plot(x,y,marker='^',color='orange',markersize=18)
        elif Nodes[i]['joint'] == 'r':
            ax.plot(x,y,marker='o',color='orange',markersize=15)
        elif Nodes[i]['joint'] == 'f':
            ax.plot(x,y,marker='s',color='orange',markersize=15)
        else:
            ax.plot(x,y,marker='s',color='k',markersize=6)
        if mr==1:
            Fx,Fy,Mz = round(Nodes[i]['external_force'][0],3),round(Nodes[i]['external_force'][1],3),round(Nodes[i]['external_force'][2],3)
        if mr==2:
            Fx,Fy,Mz = round(R[3*i,0],3),round(R[3*i+1,0],3),round(R[3*i+2,0],3)
        dx,dy = 0.5*L1,0.5*L2
        head_width = 0.2*L
        x1,y1 = head_width+1.1*dx,head_width+1.1*dy
        if Fx<0:
            ax.arrow(x+x1, y, -dx, 0, head_width = head_width, width = 0.03*L2, ec ='green')
            ax.text(x+x1, y+0.1*L2, "Fx = "+str(Fx))
        if Fx>0:
            ax.arrow(x-x1, y, dx, 0, head_width = head_width, width = 0.03*L2, ec ='green')
            ax.text(x-2*x1, y+0.1*L2, "Fx = "+str(Fx))
        if Fy<0:
            ax.arrow(x, y+y1, 0, -dy, head_width = head_width, width = 0.03*L1, ec ='green')
            ax.text(x, y+y1*1.1, "Fy = "+str(Fy))
        if Fy>0:
            ax.arrow(x, y-y1, 0, dy, head_width = head_width, width = 0.03*L1, ec ='green')
            ax.text(x, y-y1*1.1, "Fy = "+str(Fy))
        r1,r2,r3 = 0.9*L1*0.8,0.9*L2*0.8,0.45*L2*0.8
        if Mz>0 and Fy<=0:
            ellipse = Arc([x,y],r1,r2,0,0,270,color='green', linewidth=1)
            ax.add_patch(ellipse)
            ax.plot(x,y-r3,marker='>',color='green',markersize=8)
            ax.text(x, y-(r3*1.3), "Mz = "+str(Mz))
        if Mz>0 and Fy>0:
            ellipse = Arc([x,y],r1,r2,0,180,450,color='green', linewidth=1)
            ax.add_patch(ellipse)
            ax.plot(x,y+r3,marker='<',color='green',markersize=8)
            ax.text(x, y+(r3*1.3), "Mz = "+str(Mz))
        if Mz<0 and Fy<=0:
            ellipse = Arc([x,y],r1,r2,0,-90,180,color='green', linewidth=1)
            ax.add_patch(ellipse)
            ax.plot(x,y-r3,marker='<',color='green',markersize=8)
            ax.text(x, y-(r3*1.3), "Mz = "+str(Mz))
        if Mz<0 and Fy>0:
            ellipse = Arc([x,y],r1,r2,0,90,360,color='green', linewidth=1)
            ax.add_patch(ellipse)
            ax.plot(x,y+r3,marker='>',color='green',markersize=8)
            ax.text(x, y+(r3*1.3), "Mz = "+str(Mz))
            
        ### NODE ###
        ax.text(x+0.2*L1, y+0.2*L2, "N"+str(i+1))
        
        ### FORCE ###
            
    ax.set_ylim(y_min-2*L2, y_max+2*L2)
    ax.set_xlim(x_min-2*L1,x_max+2*L1)
    plt.show()

In [26]:
def main():
    ##### 2D FEM FOR TRUSS #####

    TotalNodes = int(input("Enter total number of nodes: "))
    dof = 3*TotalNodes

    ### DEFINING NODE QUANTITIES ###

    Nodes = {}
    freeNode = []
    F = []
    x_min,x_max,y_min,y_max = 1e10,-1e10,1e10,-1e10

    for i in range(TotalNodes):
        print()
        X = input("Enter (x,y) coordinates of node-"+str(i+1)+" (Example: 2,-3.7): ").split(',')
        support = input("Enter support type at node-"+str(i+1)+" ---> write 'P/p' for Planar and 'I/i' for Inclined: ")
        if support.upper() == 'P':
            angle = 0
        elif support.upper() == 'I':
            angle = float(input("Enter inclination angle in degrees (Example: 45.3): "))
        joint = input("Enter constraint type at node-"+str(i+1)+" ---> write 'f' for Fixed, 'fr' for Free, 'h' for hinged and 'r' for Roller support: ")
        if joint == 'fr':
            freeNode.append(3*i)
            freeNode.append(3*i+1)
            freeNode.append(3*i+2)
            Ext_F = input("Enter external force (Fx,Fy,Mz) at node-"+str(i+1)+" (Example: 1.2,-2,4.76 or 0,2.75,0): ").split(',')
            F.append(float(Ext_F[0]))
            F.append(float(Ext_F[1]))
            F.append(float(Ext_F[2]))
        elif joint == 'r':
            freeNode.append(3*i)
            freeNode.append(3*i+2)
            Ext_Fx = input("Enter external force (Fx,Mz) at node-"+str(i+1)+" (Example: 1.2,-3.7): ").split(',')
            Ext_F = [Ext_Fx[0],'0',Ext_Fx[1]]
            F.append(float(Ext_F[0]))
            F.append(float(Ext_F[2]))
        elif joint == 'h':
            freeNode.append(3*i+2)
            Ext_Fx = input("Enter external force Mz at node-"+str(i+1)+" (Example: 0.75): ")
            Ext_F = ['0','0',Ext_Fx]
            F.append(float(Ext_F[2]))
        else:
            Ext_F = ['0','0','0']

        Nodes[i] = {'coordinates':[float(X[0]),float(X[1])],
                    'angle': angle,
                    'joint': joint,
                    'external_force': [float(Ext_F[0]),float(Ext_F[1]),float(Ext_F[2])]}

        if x_min>Nodes[i]['coordinates'][0]:
            x_min = Nodes[i]['coordinates'][0]
        if y_min>Nodes[i]['coordinates'][1]:
            y_min = Nodes[i]['coordinates'][1]
        if x_max<Nodes[i]['coordinates'][0]:
            x_max = Nodes[i]['coordinates'][0]
        if y_max<Nodes[i]['coordinates'][1]:
            y_max = Nodes[i]['coordinates'][1]

    print()

    ### NODE ELEMENT RELATIONSHIP TABLE ###

    TotalElements = int(input("Enter total number of elements: "))

    Element = {}

    for i in range(TotalElements):
        print()
        FromTo = input("Enter (From_node, To_node) for element-"+str(i+1)+" (Example: 2,3): ").split(',')
        E = sympy.sympify(input("Enter Young's modulus of element-"+str(i+1)+" (Example: 2.27e10 if E = 2.27*10^10 or 2.7e10*(2**(0.5)) if E = 2.7*sqrt(2)*10^10): "))
        A = sympy.sympify(input("Enter area of element-"+str(i+1)+" (Example: 2.27e-3 if A = 2.27*10^-3 or 2.7*(2**(0.5)) if A = 2.7*sqrt(2)): "))
        I = sympy.sympify(input("Enter Izz of element-"+str(i+1)+" (Example: 2.27e-3 if I = 2.27*10^-3 or 2.7*(2**(0.5)) if I = 2.7*sqrt(2)): "))
        X,Y = np.array(Nodes[int(FromTo[0])-1]['coordinates']), np.array(Nodes[int(FromTo[1])-1]['coordinates'])
        L,Z = np.linalg.norm(X-Y),Y-X
        theta = np.arctan2(Z[1], Z[0])
        s,c = math.sin(theta),math.cos(theta)

        ### LOCAL STIFNESS MATRIX ###

        c1 = A*c*c + (12*I*s*s/(L*L))
        c2 = (A - (12*I/(L*L)))*c*s
        c3 = 6*I*s/L
        c4 = A*s*s + (12*I*c*c/(L*L))
        c5 = 6*I*c/L

        k = np.array([[c1, c2, -c3, -c1, -c2, -c3],
                      [c2, c4, c5, -c2, -c4, c5],
                      [-c3, c5, 4*I, c3, -c5, 2*I],
                      [-c1, -c2, c3, c1, c2, c3],
                      [-c2, -c4, -c5, c2, c4, -c5],
                      [-c3, c5, 2*I, c3, -c5, 4*I]])
        k = k*(E/L)
        Element[i] = {'s':s,
                      'c':c,
                      'L':L,
                      'A':A,
                      'E':E,
                      'I':I,
                      'k':k,
                      'From_To':[int(FromTo[0])-1,int(FromTo[1])-1]}


    ### ASSEMBLE GLOBAL STIFFNESS MATRIX ###    
    print()

    K = np.zeros((dof,dof))
    for i in range(TotalElements):
        n1,n2 = Element[i]['From_To'][0], Element[i]['From_To'][1]
        k = Element[i]['k']
        print('LOCAL STIFFNESS MATRIX k'+str(i+1)+':\n')
        print(np.around(k.astype(np.double),4))
        print()
        map_local_to_global = {0:3*n1,
                               1:3*n1+1,
                               2:3*n1+2,
                               3:3*n2,
                               4:3*n2+1,
                               5:3*n2+2}
        for i in range(6):
            for j in range(6):
                K[map_local_to_global[i],map_local_to_global[j]] += k[i,j]

    print('GLOBAL STIFFNESS MATRIX K:\n')
    print(np.around(K,4))
    print()

    ### TRANSFORM GLOBAL STIFFNESS MATRIX ###

    T = np.zeros((dof,dof))
    for i in range(TotalNodes):
        alpha = (Nodes[i]['angle'])*np.pi/180
        s,c = math.sin(alpha),math.cos(alpha)
        t = np.array([[c,s,0],
                      [-s,c,0],
                      [0,0,1]])
        T[3*i:3*i+3,3*i:3*i+3] = t

    K = (T@K)@(T.T)
    print('TRANSFORMED GLOBAL STIFFNESS MATRIX K:\n')
    print(np.around(K,4))
    print()

    ### SOLVING KQ = F ###

    K_small = np.zeros((len(freeNode),len(freeNode)))

    map_to_small = {}
    for i in range(len(freeNode)):
        map_to_small[freeNode[i]] = i

    for i in range(dof):
        if i not in freeNode:
            continue
        for j in range(dof):
            if j not in freeNode:
                continue
            K_small[map_to_small[i],map_to_small[j]] = K[i,j]

    F = np.array(F).reshape(len(freeNode),1)
    q = (np.linalg.inv(K_small))@F

    Q = np.zeros((dof,1))
    for i in range(dof):
        if i not in freeNode:
            continue
        Q[i,0] = q[map_to_small[i],0]

    print("DISPLACEMENT MATRIX Q:\n")
    print(np.around(Q,4))
    print()

    ### REACTION FORCES AND STRESSES ###

    Z = K@Q
    R = Z[:,:]
    for i in range(TotalNodes):
        if Nodes[i]['joint']=='fr':
            R[3*i,0],R[3*i+1,0],R[3*i+2,0] = 0,0,0
        elif Nodes[i]['joint']=='r':
            R[3*i,0],R[3*i+2,0] = 0,0
        elif Nodes[i]['joint']=='h':
            R[3*i+2,0] = 0
        else:
            pass

    print("REACTION FORCE R:\n")
    print(np.around(R,4))
    print()

    '''
    for i in range(TotalElements):
        n1,n2 = Element[i]['From_To'][0], Element[i]['From_To'][1]
        q = np.array([Q[2*n1,0],Q[2*n1+1,0],Q[2*n2,0],Q[2*n2+1,0]])
        c,s = Element[i]['c'],Element[i]['s']
        E,L = Element[i]['E'],Element[i]['L']
        C = np.array([-c,-s,c,s])
        S = np.dot(C,q)
        S = S*(E/L)
        if S<0:
            print("ELEMENT-"+str(i+1)+" IS UNDER COMPRESSIVE STRESS OF",S,"UNITS")
            print()
        elif S>0:
            print("ELEMENT-"+str(i+1)+" IS UNDER TENSILE STRESS OF",S,"UNITS")
            print()
        else:
            print("ELEMENT-"+str(i+1)+" IS STRESS FREE")
            print()
    '''
    Draw_truss(Nodes,Element,TotalNodes,TotalElements,x_min,x_max,y_min,y_max,R,1)
    Draw_truss(Nodes,Element,TotalNodes,TotalElements,x_min,x_max,y_min,y_max,R,2)
    print("\nAnalysis ends here!")
    
main()

Enter total number of nodes: c


ValueError: invalid literal for int() with base 10: 'c'